<a href="https://colab.research.google.com/github/velantrian/velantrim/blob/main/Velantrim_Core_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import time
from typing import List, Dict, Any

# --- КОНСТАНТЫ И НАСТРОЙКИ ---
BRAIN_PATH = 'data/brain.json'
LOG_PATH = 'data/conversation_log.json'
VELANTRIM_CULTURE = "Велантрим — культура мира и заботы о жизни. Life is dream."
LLM_MODEL = "Qwen2.5-7B-Velantrim-Tuned"
RAG_VECTOR_DB = "ChromaDB_Local"

# --- СИМУЛЯЦИЯ ФУНКЦИЙ СИСТЕМЫ ПАМЯТИ И LLM ---

def load_json(filepath: str) -> Dict[str, Any]:
    """Загружает JSON-файл (Рефлексы или Лог)."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"⚠️ Файл не найден: {filepath}. Создание пустого словаря.")
        return {}
    except json.JSONDecodeError:
        print(f"❌ Ошибка декодирования JSON в файле: {filepath}.")
        return {}

def save_log(log: List[Dict[str, str]]):
    """Сохраняет обновленный лог диалога."""
    try:
        with open(LOG_PATH, 'w', encoding='utf-8') as f:
            json.dump(log, f, ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"❌ Ошибка сохранения лога: {e}")

def simulate_rag_retrieval(query: str) -> str:
    """
    Симуляция RAG-поиска по вашей долгосрочной памяти (VectorDB).
    Использует: LlamaIndex/LangChain + ChromaDB.
    """
    print(f"🔎 Phase 2: Запуск RAG-поиска в {RAG_VECTOR_DB} по запросу: '{query}'")
    # Используем ваши личные данные для симуляции релевантного извлечения
    if "железо и витамины" in query.lower() or "диета" in query.lower():
        return "КОНТЕКСТ RAG: Пользователь плохо питался год (сухомятки, чипсы). Анализ показал, что Железо 17 µmol/l (нижняя граница) и рекомендуется коррекция питания, фокус на ЖКТ и B-витамины."
    return f"КОНТЕКСТ RAG: Извлечено 3 релевантных фрагмента знаний о Velantrim Culture и Messenger KIM."

def simulate_llm_response(full_prompt: str) -> str:
    """
    Симуляция генерации ответа с помощью LLM (Qwen2.5-7B).
    LLM получает весь контекст: Рефлексы + RAG-контекст.
    """
    print(f"💬 Phase 3: Генерация ответа через {LLM_MODEL}...")
    # Здесь LLM обрабатывает сложный промт, содержащий все данные
    if "ACTION: " in full_prompt:
        # Если сработал рефлекс, LLM просто форматирует шаблон
        action_text = full_prompt.split("ACTION: ")[1].split("\n")[0]
        return f"🤖 [ОТВЕТ РЕФЛЕКСА]: {action_text}"

    # Иначе, генерирует ответ на основе RAG-контекста
    return f"🤖 [ОТВЕТ LLM/RAG]: Анализируя ваш запрос и предоставленный контекст RAG, я могу сказать следующее: {full_prompt.split('КОНТЕКСТ RAG: ')[1]}"

# --- ГЛАВНЫЙ КЛАСС АГЕНТА VELANTRIM ---

class VelantrimCore:
    def __init__(self):
        self.brain = load_json(BRAIN_PATH)
        self.log = load_json(LOG_PATH)
        if not self.log:
            self.log = []

    def check_reflexes(self, user_query: str) -> str | None:
        """
        Phase 1: Проверяет пользовательский запрос на совпадение с Рефлексами.
        Рефлексы имеют наивысший приоритет.
        """
        print("🧠 Phase 1: Проверка механизма Рефлексов...")
        query_lower = user_query.lower()

        for reflex in self.brain.get('reflexes', []):
            keywords = reflex.get('trigger_keywords', [])

            # Проверяем на совпадение ключевых слов (базовый, но быстрый триггер)
            if any(keyword in query_lower for keyword in keywords):
                # В реальной системе здесь будет более сложная проверка 'condition'

                # Симуляция срабатывания рефлекса "Здоровье/Сон"
                if reflex['id'] == "health_sleep_osa_check":
                    template = reflex['response_template']
                    action = " ".join(template)
                    return f"REFLEX TRIGGERED (Priority {reflex['priority']}): {reflex['category']}. ACTION: {action}"

                # Симуляция срабатывания рефлекса "Предпочтения"
                if reflex['id'] == "preference_gear_recommendation" and any(item in query_lower for item in ["носки", "обувь"]):
                    brand = "Falke" if "носки" in query_lower else "Birkenstock"
                    category = "носки" if "носки" in query_lower else "обувь"
                    reason = "Прочность и посадка" if "носки" in query_lower else "Анатомический комфорт и износостойкость"

                    response = reflex['response_template'][0].format(
                        category_item=category, brand_name=brand, reason=reason
                    )
                    return f"REFLEX TRIGGERED (Priority {reflex['priority']}): {reflex['category']}. ACTION: {response}"

        return None

    def process_query(self, user_query: str):
        """Основной цикл обработки запроса."""

        # 1. Проверка Рефлексов
        reflex_result = self.check_reflexes(user_query)

        # Начало формирования промта для LLM
        llm_prompt = f"Контекст Диалога (лог последних 5 записей): {self.log[-5:] if len(self.log) > 5 else self.log}\n"
        llm_prompt += f"Культура Velantrim: {VELANTRIM_CULTURE}\n"
        llm_prompt += f"Пользовательский Запрос: {user_query}\n"

        if reflex_result:
            # Если сработал Рефлекс, не обращаемся к RAG, а сразу генерируем ответ
            llm_prompt += f"ПРИОРИТЕТНОЕ ПРАВИЛО (РЕФЛЕКС): {reflex_result}"
        else:
            # 2. RAG-поиск (только если Рефлекс не сработал)
            rag_context = simulate_rag_retrieval(user_query)
            llm_prompt += f"КОНТЕКСТ RAG: {rag_context}\n"

        # 3. Генерация финального ответа
        ai_response = simulate_llm_response(llm_prompt)

        # 4. Обновление и сохранение лога (Краткосрочная Память)
        new_entry = {
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "user": user_query,
            "ai": ai_response.split(': ', 1)[-1] # Чистый ответ без префикса
        }
        self.log.append(new_entry)
        save_log(self.log)

        print("\n--- ОТВЕТ VELANTRIM ---")
        print(ai_response)
        print("-----------------------\n")


# --- ЗАПУСК СИСТЕМЫ ---

if __name__ == "__main__":
    velantrim = VelantrimCore()

    # --- ТЕСТОВЫЕ СЦЕНАРИИ ---
    print("⚜️ СТАРТ VELANTRIM CORE ⚜️")

    # 1. Тест, который должен сработать через РЕФЛЕКС (Высокий Приоритет)
    print("\n--- СЦЕНАРИЙ 1: ТРИГГЕР РЕФЛЕКСА (Здоровье) ---")
    velantrim.process_query("Я сегодня проснулся с туманом в голове, никак не могу собраться.")

    # 2. Тест, который должен сработать через РЕФЛЕКС (Предпочтения)
    print("\n--- СЦЕНАРИЙ 2: ТРИГГЕР РЕФЛЕКСА (Предпочтения) ---")
    velantrim.process_query("Мне нужны новые носки, какие бренды я предпочитаю?")

    # 3. Тест, который должен использовать RAG (Низкий Приоритет, поиск знаний)
    print("\n--- СЦЕНАРИЙ 3: RAG-ПОИСК (Знания) ---")
    velantrim.process_query("Что мне нужно сделать, чтобы исправить проблему с низким железом и витаминами?")

    # 4. Тест, который должен сработать через РЕФЛЕКС (ЖКТ)
    print("\n--- СЦЕНАРИЙ 4: ТРИГГЕР РЕФЛЕКСА (ЖКТ) ---")
    velantrim.process_query("Думаю, куплю орехи и сухофрукты, чтобы перекусить, и пробиотик.")

    # --- СОСТОЯНИЕ ПАМЯТИ ---
    print(f"\n✨ Обновленный лог диалога ({len(velantrim.log)} записей) сохранен в {LOG_PATH}")